# PySpark DataFrame Basics

Material Bruno Rocha Oliveira adaptado por Diandra Kubo 02/2020

Links úteis:
* Spark Dataframe and Operations https://www.analyticsvidhya.com/blog/2016/10/spark-dataframe-and-operations/
* Kedro Tutorial  https://kedro.readthedocs.io/en/latest/03_tutorial/02_tutorial_template.html
* Pandas Tutorial https://towardsdatascience.com/data-handling-using-pandas-machine-learning-in-real-life-be76a697418c

## Table of Contents:
* [Select](#Select)
* [Colunas](#Colunas)
* [Filtros](#Filtros)
* [Ordenação](#Ordenação)
* [Criar campos calculados](#Criar-campos-calculados)
* [Transformação de dados](#Tranformação-de-dados)
* [Datas](#Datas)
* [Nulos](#Nulos)
* [Agrupamentos](#Agrupamentos)
* [Joins](#Joins)

Criando contexto spark a partir do contexto Kedro

In [1]:
spark = context.init_spark_session()

Importando bibliotecas que serão utilizadas

In [2]:
import pandas as pd
import numpy as np
from pyspark.sql import functions as F
import pickle
from itertools import chain

In [3]:
df = spark.table("bg.bgdtpen")

## Select

Selecionando colunas 

In [4]:
df.select (
                F.expr("substring(centro_alta, 2, 1)"),
                F.col("centro_alta"),
                F.column("centro_alta"))\
.show(2)

+----------------------------+-----------+-----------+
|substring(centro_alta, 2, 1)|centro_alta|centro_alta|
+----------------------------+-----------+-----------+
|                           8|       3871|       3871|
|                           8|       3871|       3871|
+----------------------------+-----------+-----------+
only showing top 2 rows



Selecionando colunas com alias

In [5]:
df.select (
                F.expr("centro_alta").alias("dest"),
                F.col("centro_alta").alias("destino"),
                F.column("centro_alta").alias("destino2"))\
.show(2)

+----+-------+--------+
|dest|destino|destino2|
+----+-------+--------+
|1578|   1578|    1578|
|1578|   1578|    1578|
+----+-------+--------+
only showing top 2 rows



Selecionando todas as colunas

In [6]:
df.select (F.expr("*"))\
.show(1)

+-------+-----------+------------+------+---------+------+-------+--------+------------+--------------+-------------+-----------+----------+---------+-----------+------+-----------+------------+-----------+---------+------------+-----------+--------+--------+-------+-------+------------+--------+-----------+------------+---------+-----------+-----+----------+-------------+--------+-----------+--------------+--------+----------+--------+-----+-----+-----+-----------+----------+----------+----------+-----------+--------------------+-------------+-------------+-----------------+-------------+
|entidad|centro_alta|      cuenta|divisa|numer_mov|codigo|importe|concepto|cod_impuesto|impor_impuesto|cpto_impuesto|fecha_conta|fecha_oper|hora_oper|fecha_valor|cheque|ref_interna|ind_desdobla|ind_observa|saldo_aut|entidad_cont|centro_cont|producto|subprodu|ind_aom|ind_act|ind_anulable|ind_anul|ind_car_abo|ind_giro_dep|codigo_ur|tipo_cambio|canal|ind_contab|ind_impto_mir|ind_ppal|transaccion|cod_apl

Passar uma constante 

In [7]:
from pyspark.sql.functions import lit
df.select(F.expr("centro_alta"), 
          F.col("cuenta"), 
          lit(1).alias("One")).show(2)

+-----------+------------+---+
|centro_alta|      cuenta|One|
+-----------+------------+---+
|       0805|000010000198|  1|
|       0805|000010000198|  1|
+-----------+------------+---+
only showing top 2 rows



## Colunas

Adicionando colunas no dataframe 

In [8]:
df.withColumn("numberOne", lit(1)).show(2)

+-------+-----------+------------+------+---------+------+-------+--------+------------+--------------+-------------+-----------+----------+---------+-----------+------+-----------+------------+-----------+---------+------------+-----------+--------+--------+-------+-------+------------+--------+-----------+------------+---------+-----------+-----+----------+-------------+--------+-----------+--------------+--------+----------+--------+-----+-----+-----+-----------+----------+----------+----------+-----------+--------------------+-------------+-------------+-----------------+-------------+---------+
|entidad|centro_alta|      cuenta|divisa|numer_mov|codigo|importe|concepto|cod_impuesto|impor_impuesto|cpto_impuesto|fecha_conta|fecha_oper|hora_oper|fecha_valor|cheque|ref_interna|ind_desdobla|ind_observa|saldo_aut|entidad_cont|centro_cont|producto|subprodu|ind_aom|ind_act|ind_anulable|ind_anul|ind_car_abo|ind_giro_dep|codigo_ur|tipo_cambio|canal|ind_contab|ind_impto_mir|ind_ppal|transacci

In [9]:
df.withColumn("target_centro_alta", 
            F.expr("centro_alta == '0001'"))\
.show(2)

+-------+-----------+------------+------+---------+------+--------+--------+------------+--------------+-------------+-----------+----------+---------+-----------+------+---------------+------------+-----------+----------+------------+-----------+--------+--------+-------+-------+------------+--------+-----------+------------+---------+-----------+-----+----------+-------------+--------+-----------+--------------+--------+----------+--------+-----+-----+-----+-----------+----------+----------+----------+-----------+--------------------+-------------+-------------+-----------------+-------------+------------------+
|entidad|centro_alta|      cuenta|divisa|numer_mov|codigo| importe|concepto|cod_impuesto|impor_impuesto|cpto_impuesto|fecha_conta|fecha_oper|hora_oper|fecha_valor|cheque|    ref_interna|ind_desdobla|ind_observa| saldo_aut|entidad_cont|centro_cont|producto|subprodu|ind_aom|ind_act|ind_anulable|ind_anul|ind_car_abo|ind_giro_dep|codigo_ur|tipo_cambio|canal|ind_contab|ind_impto_m

Renomeando colunas

In [10]:
df.withColumnRenamed("centro_alta", "agencia").columns

['entidad',
 'agencia',
 'cuenta',
 'divisa',
 'numer_mov',
 'codigo',
 'importe',
 'concepto',
 'cod_impuesto',
 'impor_impuesto',
 'cpto_impuesto',
 'fecha_conta',
 'fecha_oper',
 'hora_oper',
 'fecha_valor',
 'cheque',
 'ref_interna',
 'ind_desdobla',
 'ind_observa',
 'saldo_aut',
 'entidad_cont',
 'centro_cont',
 'producto',
 'subprodu',
 'ind_aom',
 'ind_act',
 'ind_anulable',
 'ind_anul',
 'ind_car_abo',
 'ind_giro_dep',
 'codigo_ur',
 'tipo_cambio',
 'canal',
 'ind_contab',
 'ind_impto_mir',
 'ind_ppal',
 'transaccion',
 'cod_aplicacion',
 'term_nio',
 'fecha_nio',
 'hora_nio',
 'mov30',
 'mov60',
 'mov90',
 'entidad_umo',
 'centro_umo',
 'userid_umo',
 'cajero_umo',
 'netname_umo',
 'timest_umo',
 'discriminante',
 'div_contraria',
 'tx_comp_hist_lanc',
 'dat_ref_carga']

Removendo colunas

In [11]:
df.drop("concepto").columns

['entidad',
 'centro_alta',
 'cuenta',
 'divisa',
 'numer_mov',
 'codigo',
 'importe',
 'cod_impuesto',
 'impor_impuesto',
 'cpto_impuesto',
 'fecha_conta',
 'fecha_oper',
 'hora_oper',
 'fecha_valor',
 'cheque',
 'ref_interna',
 'ind_desdobla',
 'ind_observa',
 'saldo_aut',
 'entidad_cont',
 'centro_cont',
 'producto',
 'subprodu',
 'ind_aom',
 'ind_act',
 'ind_anulable',
 'ind_anul',
 'ind_car_abo',
 'ind_giro_dep',
 'codigo_ur',
 'tipo_cambio',
 'canal',
 'ind_contab',
 'ind_impto_mir',
 'ind_ppal',
 'transaccion',
 'cod_aplicacion',
 'term_nio',
 'fecha_nio',
 'hora_nio',
 'mov30',
 'mov60',
 'mov90',
 'entidad_umo',
 'centro_umo',
 'userid_umo',
 'cajero_umo',
 'netname_umo',
 'timest_umo',
 'discriminante',
 'div_contraria',
 'tx_comp_hist_lanc',
 'dat_ref_carga']

Cast colunas 

In [12]:
df.withColumn("agencia_int", F.col("centro_alta").cast("integer"))

DataFrame[entidad: string, centro_alta: string, cuenta: string, divisa: string, numer_mov: decimal(11,0), codigo: string, importe: decimal(17,2), concepto: string, cod_impuesto: string, impor_impuesto: decimal(17,2), cpto_impuesto: string, fecha_conta: string, fecha_oper: string, hora_oper: string, fecha_valor: string, cheque: decimal(11,0), ref_interna: string, ind_desdobla: string, ind_observa: string, saldo_aut: decimal(17,2), entidad_cont: string, centro_cont: string, producto: string, subprodu: string, ind_aom: string, ind_act: string, ind_anulable: string, ind_anul: string, ind_car_abo: string, ind_giro_dep: string, codigo_ur: string, tipo_cambio: decimal(11,6), canal: string, ind_contab: string, ind_impto_mir: string, ind_ppal: string, transaccion: string, cod_aplicacion: string, term_nio: string, fecha_nio: string, hora_nio: string, mov30: decimal(17,2), mov60: decimal(17,2), mov90: decimal(17,2), entidad_umo: string, centro_umo: string, userid_umo: string, cajero_umo: string, 

## Filtros

Filtrando registros

In [13]:
df.filter(F.col("importe") < 100).show(2)

+-------+-----------+------------+------+---------+------+--------+--------+------------+--------------+-------------+-----------+----------+---------+-----------+------+-----------+------------+-----------+---------+------------+-----------+--------+--------+-------+-------+------------+--------+-----------+------------+---------+-----------+-----+----------+-------------+--------+-----------+--------------+--------+----------+--------+-----+-----+-----+-----------+----------+----------+----------+-----------+--------------------+-------------+-------------+--------------------+-------------+
|entidad|centro_alta|      cuenta|divisa|numer_mov|codigo| importe|concepto|cod_impuesto|impor_impuesto|cpto_impuesto|fecha_conta|fecha_oper|hora_oper|fecha_valor|cheque|ref_interna|ind_desdobla|ind_observa|saldo_aut|entidad_cont|centro_cont|producto|subprodu|ind_aom|ind_act|ind_anulable|ind_anul|ind_car_abo|ind_giro_dep|codigo_ur|tipo_cambio|canal|ind_contab|ind_impto_mir|ind_ppal|transaccion|co

Varios filtros 

```
SELECT * 
FROM df 
WHERE importe < 100 
AND dat_ref_carga == "2020-01-06"
AND centro_alta == '0002'
LIMIT 2
```

In [14]:
df = df.where(F.col("importe") < 100)\
       .where(F.col("dat_ref_carga") == "2020-01-06")\
       .where(F.col("centro_alta") == "0002")
df.show(2)

+-------+-----------+------------+------+---------+------+-------+--------+------------+--------------+-------------+-----------+----------+---------+-----------+------+-----------+------------+-----------+---------+------------+-----------+--------+--------+-------+-------+------------+--------+-----------+------------+---------+-----------+-----+----------+-------------+--------+-----------+--------------+--------+----------+--------+-----+-----+-----+-----------+----------+----------+----------+-----------+--------------------+-------------+-------------+--------------------+-------------+
|entidad|centro_alta|      cuenta|divisa|numer_mov|codigo|importe|concepto|cod_impuesto|impor_impuesto|cpto_impuesto|fecha_conta|fecha_oper|hora_oper|fecha_valor|cheque|ref_interna|ind_desdobla|ind_observa|saldo_aut|entidad_cont|centro_cont|producto|subprodu|ind_aom|ind_act|ind_anulable|ind_anul|ind_car_abo|ind_giro_dep|codigo_ur|tipo_cambio|canal|ind_contab|ind_impto_mir|ind_ppal|transaccion|cod_

Filtro OR
```
SELECT *
FROM df
WHERE importe<10 or importe>90
```

In [15]:
df.where((F.col("importe")<10) | (F.col("importe")>90)).show(2)

+-------+-----------+------------+------+---------+------+-------+--------+------------+--------------+-------------+-----------+----------+---------+-----------+------+-----------+------------+-----------+---------+------------+-----------+--------+--------+-------+-------+------------+--------+-----------+------------+---------+-----------+-----+----------+-------------+--------+-----------+--------------+--------+----------+--------+-----+-----+-----+-----------+----------+----------+----------+-----------+--------------------+-------------+-------------+--------------------+-------------+
|entidad|centro_alta|      cuenta|divisa|numer_mov|codigo|importe|concepto|cod_impuesto|impor_impuesto|cpto_impuesto|fecha_conta|fecha_oper|hora_oper|fecha_valor|cheque|ref_interna|ind_desdobla|ind_observa|saldo_aut|entidad_cont|centro_cont|producto|subprodu|ind_aom|ind_act|ind_anulable|ind_anul|ind_car_abo|ind_giro_dep|codigo_ur|tipo_cambio|canal|ind_contab|ind_impto_mir|ind_ppal|transaccion|cod_

Distinct
```
SELECT COUNT(DISTINCT(centro_alta, cuenta)) FROM df
```

In [16]:
df.select("centro_alta", "cuenta").distinct().count()

14710

Union

In [17]:
df_aux = spark.table("bg.bgdtpen")\
        .filter(F.col("dat_ref_carga") == "2020-02-03")\
        .filter(F.col("centro_alta") == '0001')

In [18]:
df.count(), df_aux.count()

(51109, 121065)

In [19]:
union = df.union(df_aux)

In [20]:
union.count()

172174

## Ordenação

Ordenar as linhas

In [21]:
df.sort("importe").show(5)

+-------+-----------+------------+------+---------+------+----------+--------+------------+--------------+-------------+-----------+----------+---------+-----------+------+-----------+------------+-----------+----------+------------+-----------+--------+--------+-------+-------+------------+--------+-----------+------------+---------+-----------+-----+----------+-------------+--------+-----------+--------------+--------+----------+--------+-----+-----+-----+-----------+----------+----------+----------+-----------+--------------------+-------------+-------------+--------------------+-------------+
|entidad|centro_alta|      cuenta|divisa|numer_mov|codigo|   importe|concepto|cod_impuesto|impor_impuesto|cpto_impuesto|fecha_conta|fecha_oper|hora_oper|fecha_valor|cheque|ref_interna|ind_desdobla|ind_observa| saldo_aut|entidad_cont|centro_cont|producto|subprodu|ind_aom|ind_act|ind_anulable|ind_anul|ind_car_abo|ind_giro_dep|codigo_ur|tipo_cambio|canal|ind_contab|ind_impto_mir|ind_ppal|transacc

In [22]:
df.orderBy("centro_alta", "cuenta").show(5)

+-------+-----------+------------+------+---------+------+-------+--------+------------+--------------+-------------+-----------+----------+---------+-----------+------+-----------+------------+-----------+---------+------------+-----------+--------+--------+-------+-------+------------+--------+-----------+------------+---------+-----------+-----+----------+-------------+--------+-----------+--------------+--------+----------+--------+-----+-----+-----+-----------+----------+----------+----------+-----------+--------------------+-------------+-------------+--------------------+-------------+
|entidad|centro_alta|      cuenta|divisa|numer_mov|codigo|importe|concepto|cod_impuesto|impor_impuesto|cpto_impuesto|fecha_conta|fecha_oper|hora_oper|fecha_valor|cheque|ref_interna|ind_desdobla|ind_observa|saldo_aut|entidad_cont|centro_cont|producto|subprodu|ind_aom|ind_act|ind_anulable|ind_anul|ind_car_abo|ind_giro_dep|codigo_ur|tipo_cambio|canal|ind_contab|ind_impto_mir|ind_ppal|transaccion|cod_

In [23]:
df.orderBy(F.col("centro_alta"), F.col("cuenta")).show(5)

+-------+-----------+------------+------+---------+------+-------+--------+------------+--------------+-------------+-----------+----------+---------+-----------+------+-----------+------------+-----------+---------+------------+-----------+--------+--------+-------+-------+------------+--------+-----------+------------+---------+-----------+-----+----------+-------------+--------+-----------+--------------+--------+----------+--------+-----+-----+-----+-----------+----------+----------+----------+-----------+--------------------+-------------+-------------+--------------------+-------------+
|entidad|centro_alta|      cuenta|divisa|numer_mov|codigo|importe|concepto|cod_impuesto|impor_impuesto|cpto_impuesto|fecha_conta|fecha_oper|hora_oper|fecha_valor|cheque|ref_interna|ind_desdobla|ind_observa|saldo_aut|entidad_cont|centro_cont|producto|subprodu|ind_aom|ind_act|ind_anulable|ind_anul|ind_car_abo|ind_giro_dep|codigo_ur|tipo_cambio|canal|ind_contab|ind_impto_mir|ind_ppal|transaccion|cod_

## Criar campos calculados

In [24]:
fabricatedQuantity = F.pow(F.col("importe") * F.col("saldo_aut"), 2) + 5

In [25]:
df.select(F.expr("cuenta"), 
          fabricatedQuantity.alias("new_quantity")).show(2)

+------------+-----------------+
|      cuenta|     new_quantity|
+------------+-----------------+
|000010005848|2.488626141449E12|
|000010008298|  3.5175752709E10|
+------------+-----------------+
only showing top 2 rows



## Tranformação de dados

Coalesce

In [26]:
df_null = df.select(F.lit(None).alias("colunaX").cast("integer"), 
          F.col("importe"), 
          F.col("cuenta"), 
          F.coalesce(F.lit(None), F.col("importe"), F.col("cuenta")))\
        .limit(5)
df_null.show()

+-------+-------+------------+-------------------------------+
|colunaX|importe|      cuenta|coalesce(NULL, importe, cuenta)|
+-------+-------+------------+-------------------------------+
|   null|-200.00|000010005848|                        -200.00|
|   null|-400.00|000010008298|                        -400.00|
|   null|-200.00|000010008298|                        -200.00|
|   null|-404.80|000010008717|                        -404.80|
|   null|-285.76|000010008717|                        -285.76|
+-------+-------+------------+-------------------------------+



Arredondar dados

In [27]:
df.select(F.round(F.lit("2.5")), F.bround(F.lit("2.5"))).show(2)

+-------------+--------------+
|round(2.5, 0)|bround(2.5, 0)|
+-------------+--------------+
|          3.0|           2.0|
|          3.0|           2.0|
+-------------+--------------+
only showing top 2 rows



Converter string to date 

In [28]:
df.withColumn("date", F.lit("2019-01-01"))\
    .select(F.to_date(F.col("date"))).show(1)

+---------------+
|to_date(`date`)|
+---------------+
|     2019-01-01|
+---------------+
only showing top 1 row



## Datas

Adicionando e subtraindo dias

In [29]:
df.select(F.col("fecha_conta"),
          F.date_sub(F.col("fecha_conta"), 5), 
          F.date_add(F.col("fecha_conta"), 5)).show(1)

+-----------+------------------------+------------------------+
|fecha_conta|date_sub(fecha_conta, 5)|date_add(fecha_conta, 5)|
+-----------+------------------------+------------------------+
| 2020-01-03|              2019-12-29|              2020-01-08|
+-----------+------------------------+------------------------+
only showing top 1 row



Diferença de dias entre datas

In [30]:
df.withColumn("week_ago", F.date_sub(F.col("fecha_conta"), 7))\
    .select(F.datediff(F.col("fecha_conta"), F.col("week_ago"))).show(1)

+-------------------------------+
|datediff(fecha_conta, week_ago)|
+-------------------------------+
|                              7|
+-------------------------------+
only showing top 1 row



Diferença de meses entre datas

In [31]:
df.select(
    F.to_date(lit("2016-01-01")).alias("start"),
    F.to_date(lit("2017-05-01")).alias("end"))\
.select(F.months_between(F.col("end"), F.col("start"))).show(1)

+--------------------------------+
|months_between(end, start, true)|
+--------------------------------+
|                            16.0|
+--------------------------------+
only showing top 1 row



## Nulos

Tirando valores nulos

In [32]:
df_null.na.drop("all", subset=["importe", "cuenta"]).show()

+-------+-------+------------+-------------------------------+
|colunaX|importe|      cuenta|coalesce(NULL, importe, cuenta)|
+-------+-------+------------+-------------------------------+
|   null|-200.00|000010005848|                        -200.00|
|   null|-400.00|000010008298|                        -400.00|
|   null|-200.00|000010008298|                        -200.00|
|   null|-404.80|000010008717|                        -404.80|
|   null|-285.76|000010008717|                        -285.76|
+-------+-------+------------+-------------------------------+



In [33]:
df_null.na.drop("all", subset=["colunaX"]).show()

+-------+-------+------+-------------------------------+
|colunaX|importe|cuenta|coalesce(NULL, importe, cuenta)|
+-------+-------+------+-------------------------------+
+-------+-------+------+-------------------------------+



Preenchendo nulos

In [34]:
df_null.na.fill(0, subset=["colunaX"]).show()

+-------+-------+------------+-------------------------------+
|colunaX|importe|      cuenta|coalesce(NULL, importe, cuenta)|
+-------+-------+------------+-------------------------------+
|      0|-200.00|000010005848|                        -200.00|
|      0|-400.00|000010008298|                        -400.00|
|      0|-200.00|000010008298|                        -200.00|
|      0|-404.80|000010008717|                        -404.80|
|      0|-285.76|000010008717|                        -285.76|
+-------+-------+------------+-------------------------------+



## Agrupamentos

```
SELECT cuenta, importe, count(*)
FROM df
GROUP BY cuenta, importe
```

In [35]:
df.groupBy("cuenta", "importe").count().show()

+------------+-------+-----+
|      cuenta|importe|count|
+------------+-------+-----+
|000010044702|  -0.07|    1|
|000010128664| -43.68|    1|
|000010141139| -50.00|    1|
|000010167793|-324.19|    1|
|000010171260| -29.50|    1|
|000010185988|  -8.85|    1|
|000010185995|-110.00|    1|
|000010203424| -35.00|    1|
|000010210097| -10.00|    1|
|000010223020| -24.00|    1|
|000010242797| -65.50|    1|
|000010264986| -22.00|    3|
|000010268117|-486.69|    1|
|000010270833|-263.45|    1|
|000010274851|  -6.00|    1|
|000010281589|  -0.14|    1|
|000010286759|  -0.18|    1|
|000010292871|  -1.24|    1|
|000010296291| -50.00|    1|
|000010299483|  -4.96|    1|
+------------+-------+-----+
only showing top 20 rows



```
SELECT cuenta, 
       avg(importe) as importe_medio, 
       stdev(saldo_aut) as saldo_std, 
       count(*) as count
FROM df
GROUP BY cuenta
```

In [36]:
df.groupBy("cuenta")\
    .agg(F.expr("avg(importe)").alias("importe_medio"),
         F.expr("stddev(saldo_aut)").alias("saldo_std"),
         F.expr("count(*)").alias("count")
    )\
.show()

+------------+-------------+------------------+-----+
|      cuenta|importe_medio|         saldo_std|count|
+------------+-------------+------------------+-----+
|000010026902|   -47.600000|               NaN|    1|
|000010038680|   -45.877500|61.970468437259115|    4|
|000010051014| -1333.996667|1.1159002344893303|    3|
|000010114427|    -1.855000|0.6293250352560177|    2|
|000010155699|   -38.021250| 92.33271185539488|    8|
|000010168127|    24.825000|15.096729778332726|    2|
|000010186006|  -670.000000|14.142135623730951|    2|
|000010234484|   -91.431250|284.97924676318854|    8|
|000010243774|  -196.960476| 1324.196727159165|   21|
|000010256219|  -186.102857|415.20596112910357|    7|
|000010441714|   -98.202000|122.97868994260755|    5|
|000010442849|    87.500000|56.568542494923804|    2|
|000010515309|   -23.642000| 59.92607469021963|   10|
|000010519633|    -6.367600|58.502390825788694|   25|
|000010615980|   -99.900000|               NaN|    1|
|000010621167| -2978.880000|

## Joins

In [37]:
df_aux = spark.table("bg.bgdtcoo")\
            .filter(F.col("dat_ref_carga")=='2020-01-06')

```
SELECT a.*, b.alfa_chica, b.alfa_grande 
FROM bg.bgdtpen a
INNER JOIN bg.bgdtcoo b 
    ON a.codigo=b.codigo
```

In [38]:
join = df.alias("a").join(df_aux.alias("b"), 
                          on="codigo",
                          how="inner")\
       .select("a.*", F.col("b.alfa_chica"), F.col("alfa_grande"))

In [39]:
join.select("codigo", "alfa_grande").show(1)

+------+--------------------+
|codigo|         alfa_grande|
+------+--------------------+
|  2162|PAGAMENTO DE CARN...|
+------+--------------------+
only showing top 1 row



Uma vez que tenho dados sumarizados, posso passar para dataframe pandas

In [41]:
df_pandas = join.groupby("alfa_grande").count().toPandas()

In [42]:
df_pandas

,alfa_grande,count
0,ESTORNO DE SEGURO,1
1,PAGAMENTO CONTA CELULAR EM CANAIS,233
2,PAGAMENTO DE TITULOS-BCE,54
3,TAR DEBITO AUTOMATICO EFETIVADO,1
4,LIQUIDO DE VENCIMENTO,31
...,...,...
214,COMISSAO RECURSO NAO DISP. DIA,2
215,SAQUE AVULSO PARA PAGAMENTOS,2
216,TARIFA MENSALIDADE PACOTE SERVICOS,157
217,CR COB BLOQ COMP CONF RECEBIMENTO,2
